# Robot Framework Expert Fine-Tune (Unsloth)

This notebook fine-tunes a small instruct model using QLoRA and pushes the adapter to Hugging Face.

In [ ]:
# Dependencies are installed in the repo setup cell above.


In [ ]:
import os, subprocess, json
repo_url = "{{REPO_URL}}"
if not repo_url:
    raise SystemExit('REPO_URL is missing')
!git clone {repo_url} repo
%cd repo
print('Repo cloned')

In [ ]:
import os
from getpass import getpass

token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN')
try:
    from google.colab import userdata
    token = token or userdata.get('HF_TOKEN')
except Exception:
    pass

if not token:
    token = getpass('Enter your Hugging Face token (HF_TOKEN): ')
os.environ['HF_TOKEN'] = token
print('HF_TOKEN set')


In [ ]:
from pathlib import Path
from datasets import load_dataset
dataset_repo = "{{DATASET_REPO}}"
train_path = Path('dataset/train.jsonl')
eval_path = Path('dataset/eval.jsonl')
if train_path.exists() and eval_path.exists():
    print('Using local dataset')
elif dataset_repo:
    print(f'Downloading dataset {dataset_repo}')
    ds = load_dataset(dataset_repo)
    Path('dataset').mkdir(exist_ok=True)
    ds['train'].to_json(train_path, orient='records', lines=True)
    ds['test' if 'test' in ds else 'validation' if 'validation' in ds else 'train'].to_json(eval_path, orient='records', lines=True)
else:
    print('Building dataset locally (skip validation)')
    !python tools/build_dataset.py --skip-validation

In [ ]:
base_model = "{{BASE_MODEL}}"
fallback_model = "{{FALLBACK_MODEL}}"
hf_repo = "{{HF_REPO}}"
seed = {{SEED}}
!python train/train_unsloth.py --base-model {base_model} --fallback-model {fallback_model} --hf-repo {hf_repo} --seed {seed}